A notebook for capturing dark frames for the calibration data for the cameras of the OROCHI Simualtor of the Kameda Lab.

550 nm illumination
Apertures set to F/16
Lamp set to 2.

Eigth round of experiments after setting bias to 4 DN.

# Process Overview

This notebook controls LOROS to capture the dark frames to accompany the radiometric calibration data captured on 21/1/2024.

# Camera Setup

Here we configure and setup the camera capture objects.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import orochi_sim_ctrl as osc

ic = osc.start_ic()
camera_config = osc.load_camera_config(cwl=550, fwhm=25, fnumber=16)
cameras = osc.connect_cameras(ic, camera_config)
osc.configure_cameras(cameras) # updated bias to 4 DN

In [ ]:
session = 'rikkyo_radiometric_calibration_darkframes_22012024'

In [ ]:
osc.set_camera_session(cameras, session)

In [ ]:
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from time import sleep

# Calibration Functions

Here we define simple functions for controlling the cameras during the calibration routines.

In [ ]:
def camera_calibration_sequence(camera: osc.Channel, scene) -> pd.DataFrame:
    """Capture pairs of images with increasing exposure time until all
    pixels are saturated, then decrease exposure time until no pixels are 
    saturated.
    
    Args:
        camera: Camera object to capture images from.
    
    Returns:
        DataFrame with columns:
            - exposure
            - mean
            - std
    """
    camera.set_property('Exposure', 'Auto', 0, 'Switch')        
    t_min = 1.0/16666 # (seconds) verify that this is the minimum exposure time
    t_max = 5.0
    t_exp = t_min
    up_scale = 1.05
    dwn_scale = 0.99
    t_scale = up_scale
    img_type='calibration'
    stop = False    
    cmd_exposures = []
    exposures = []
    means = []
    stds = []
    shots = []
    sat_val = camera.max_dn
    while stop is False:
        print(f'Exposure: {t_exp:.7f} s')
        # camera.set_property('Exposure', 'Value', t_exp, 'AbsoluteValue')
        camera.set_exposure(t_exp)
        img_1 = camera.image_capture(roi=True)
        # sleep(0.5)
        img_2 = camera.image_capture(roi=True)
        # sleep(0.5)
        name_1 = f'{t_exp*1E6:.0f}us_1'
        name_2 = f'{t_exp*1E6:.0f}us_2'
        camera.save_image(name_1, img_type, img_1)
        camera.save_image(name_2, img_type, img_2)
        exposures.append(camera.get_exposure_value())
        cmd_exposures.append(t_exp)
        mean = np.mean([np.mean(img_1), np.mean(img_2)])
        std = np.mean([np.std(img_1), np.std(img_2)])
        shot = np.sqrt(np.mean((img_1 - img_2)**2)/2)
        means.append(mean)
        stds.append(std)
        shots.append(shot)
        if mean >= sat_val * 0.999:
            t_scale = dwn_scale
        if t_scale == dwn_scale:
            # n_sat = (np.sum(img_1 == sat_val) + np.sum(img_2 == sat_val)) / 2
            if mean <= sat_val * 0.80:
                stop = True
        else:
            if t_exp*t_scale > t_max:
                t_scale = dwn_scale
        t_exp = t_exp * t_scale

    results = pd.DataFrame({'exposure': exposures, 'raw mean': means, 'raw standard deviation': stds, 'shot and read noise': shots, 'cmd exposure': cmd_exposures})
    results.sort_values('exposure', inplace=True)

    print(results)

    cwl_str = str(int(camera.camera_props['cwl']))
    channel = str(camera.camera_props['number'])+'_'+cwl_str
    subject_dir = Path('..', '..', 'data', 'sessions', camera.session, camera.scene, channel)
    subject_dir.mkdir(parents=True, exist_ok=True)
    filename = 'exposures.csv'
    img_file =str(Path(subject_dir, filename).with_suffix('.tif'))

    results['exposure'].to_csv(f'{camera.scene}_exposures.csv', index=False)

    fig, ax = plt.subplots(2,1, figsize=(5,10))
    ax[0].plot(results['exposure'], results['raw mean'], label='Ret.')
    ax[0].plot(results['cmd exposure'], results['raw mean'], label='Cmd.')
    ax[0].set_xlabel('Exposure (s)')
    ax[0].set_ylabel('Mean Pixel Value')
    results.sort_values('raw mean', inplace=True)
    ax[1].loglog(results['raw mean'], results['raw standard deviation'], label='Total Noise')
    ax[1].loglog(results['raw mean'], results['shot and read noise'], label='Shot and Read Noise')
    ax[1].set_xlabel('Mean Pixel Value')
    ax[1].set_ylabel('Standard Deviation')
    plt.show()
    return cmd_exposures

# Dark Frame Capture

Output:
- addition of dark mean frames to SCT99 directory
- addition of dark mean frames to SCT5 directory

In [ ]:
osc.set_camera_scene(cameras, 'SCT99_lamp2_dark_21012024')

In [ ]:
channel2camera = {}

for camera in cameras:
    channel2camera[f"{camera.camera_props['number']}_{camera.camera_props['cwl']}"] = camera.name

In [ ]:
channel2camera

# find the exposure used before

In [ ]:
import orochi_sim_proc as osp
import os

light_path = Path('..','..', 'data', 'sessions', 'rikkyo_radiometric_calibration_22012024', 'SCT99_lamp2_21012024')

channels = sorted(list(next(os.walk(light_path))[1]))  
lamp2_exposures = {}
for channel in channels:  
    frame_1s = sorted(list(Path(light_path, channel).glob('[!.]*_1_calibration.tif')))
    n_steps = len(frame_1s)
    exposures = []
    for i in range(n_steps):
        img_1 = osp.Image(light_path, None, channel, img_type='img')
        img_1.image_load(filename=frame_1s[i].name)
        exposures.append(float(img_1.exposure))
    lamp2_exposures[channel2camera[channel]] = exposures

In [ ]:
lamp2_exposures

In [ ]:
t_min = 1.0/16666

In [ ]:
for camera in cameras:
    exposures = lamp2_exposures[camera.name]
    for exposure in exposures:
        if exposure < t_min:
            exposure = t_min
        # camera.set_property('Exposure', 'Value', exposure, 'AbsoluteValue')
        camera.set_exposure(exposure)
        drk = camera.image_capture(roi=True)
        for i in range(4):
            drk += camera.image_capture(roi=True)
        drk_ave = drk.astype(np.float32)/5
        name = f'{exposure*1E6:.0f}us_d'
        camera.save_image(name, 'drk', drk_ave)

In [ ]:
osc.set_camera_scene(cameras, 'SCT99_lamp10_dark_21012024')

In [ ]:
import orochi_sim_proc as osp
import os

light_path = Path('..','..', 'data', 'sessions', 'rikkyo_radiometric_calibration_22012024', 'SCT99_lamp10_21012024')

channels = sorted(list(next(os.walk(light_path))[1]))  
lamp10_exposures = {}
for channel in channels:  
    frame_1s = sorted(list(Path(light_path, channel).glob('[!.]*_1_calibration.tif')))
    n_steps = len(frame_1s)
    exposures = []
    for i in range(n_steps):
        img_1 = osp.Image(light_path, None, channel, img_type='img')
        img_1.image_load(filename=frame_1s[i].name)
        exposures.append(float(img_1.exposure))
    lamp10_exposures[channel2camera[channel]] = exposures

In [ ]:
osc.set_camera_scene(cameras, 'SCT99_lamp10_dark_21012024')

In [ ]:
for camera in cameras:
    exposures = lamp10_exposures[camera.name]
    for exposure in exposures:
        if exposure < t_min:
            exposure = t_min
        # camera.set_property('Exposure', 'Value', exposure, 'AbsoluteValue')
        camera.set_exposure(exposure)
        drk = camera.image_capture(roi=True)
        for i in range(4):
            drk += camera.image_capture(roi=True)
        drk_ave = drk.astype(np.float32)/5
        name = f'{exposure*1E6:.0f}us_d'
        camera.save_image(name, 'drk', drk_ave)